In [ ]:
# Basic
import openai

client = openai.OpenAI()

response = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[{"role": "user", "content": "왜 하늘은 파란거야?"}],
    reasoning_effort="minimal",
    verbosity="low",
    # n=8,
)

# for choice in response.choices:
#     print(choice)

message = response.choices[0].message.content
message

In [ ]:
# Memories
import openai
import json

client = openai.OpenAI()
messages = []

# Tool
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather for",
                    }
                },
                "required": ["city"],
            },
        },
    }
]


# Dummy function that called by tool
def get_weather(city: str) -> str:
    return f"The current weather in {city} is sunny with a temperature of 25°C."


FUNCTION_MAP = {"get_weather": get_weather}


def process_ai_response(message):
    if message.tool_calls:  # 사용할 Tool 있는지 확인
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [  # Tool 호출 정보
                    {
                        "id": tool_call.id,
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                        "type": tool_call.type,
                    }
                    for tool_call in message.tool_calls
                ],
            }
        )

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments
            print(f"Calling function: {function_name} with arguments: {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(
                function_name
            )  # 호출해야 할 함수 가져오기
            result = (
                function_to_run(**arguments) if function_to_run else None
            )  # `**` 는 딕셔너리를 언팩킹하는 역할
            print(f"Ran {function_name} with args: {arguments} for result of {result}")

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": result,
                }
            )  # Tool 호출 결과 추가

        call_ai()  # 사용해야 하는 Tool 이 있는지 다시 한번 AI 를 호출하여 확인
    else:  # Tool 사용 하지 않을 때
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI : {message.content}")


def call_ai() -> None:
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=messages,
        reasoning_effort="minimal",
        verbosity="low",
        tools=TOOLS,  # type: ignore
    )
    process_ai_response(
        response.choices[0].message
    )  # messages 와 tool_schema를 함께 전달
    # message = response.choices[0].message.content # Duplicated context
    # messages.append({"role": "assistant", "content": message})
    # print("AI (call_ai): ", message)


while True:
    message = input("Send a message")
    if message == "quit" or message == "exit":
        break
    else:
        messages.append({"role": "user", "content": message})
        print("User:", message)
        call_ai()